# Explainer notebook for Copenhagen visualizations by Jamie and Valentin

Below is the code and some static graphs used for processing the data used in our project for exploring the city of Copenhagen.
The code is split into the sections that roughly corresponds to the website.

N.B. the code will generate files from the relative path "src/data/" so remember to have the whole project folder downloaded to avoid errors.

## Imports, setup and Introduction
We'll import a data set here that is based on peoples age, their income and the district they live in.

In [ ]:
import os, collections, csv
import pandas as pd, matplotlib as mat, matplotlib.pyplot as plt
root_directory = os.path.abspath(os.pardir)
df = pd.read_csv(os.path.join(root_directory, "src","data","kon_alder_bydel_penge.csv"), encoding="utf-8") # reads csv

# Show number of people in each district
print df.groupby(['Bydel'])['Personer'].sum()

Looks good.
We know that there are many tall buildings on Nørrebro and more villas and Brønshøj-Husum

Moving on

## People in age groups listed by districts
Let's try to see the distribution here.
A lot of data processing to calculate the ratio between age groups in different districts.

In [ ]:
%matplotlib inline
# -*- coding: utf-8 -*-

Districts = df.Bydel.unique()
age_categories = df.Aldersgruppe.unique()

# Total number of people, and people in an age category
total_people = df['Personer'].sum()
people_in_age_category = df.groupby(['Aldersgruppe'])['Personer'].sum().astype(float)

# Create dictionary with ratios
age_category_sum = []
for category in age_categories:
    age_category_sum.append((category, (people_in_age_category[category] / total_people)))
age_category_sum_dict = dict(age_category_sum)


# Three subplots sharing both x/y axes
f, axarr = plt.subplots(5,2, sharey=True, sharex=True, figsize=(10,10))

# Loop through districts and calculate ratio of people living in a district by their age category
index = 0
jj = 0
people_in_districts_as_ratio = {}
for district in Districts:
    people_in_districts_as_ratio[district] = {}
    people_in_district = df[df['Bydel'].isin([district])].groupby(['Aldersgruppe'])['Personer'].sum().astype(float)
    total_people_in_district = people_in_district.sum()
    for category in age_categories:
        try:
            people_in_districts_as_ratio[district][category] = (people_in_district[category] / total_people_in_district)
        except KeyError:
            people_in_districts_as_ratio[district][category] = 0
    
    d_temp = collections.OrderedDict(sorted(people_in_districts_as_ratio[district].items()))
    axarr[index%5, index%2].set_title(district)
    h = axarr[index%5, index%2].bar(range(len(d_temp)), d_temp.values(), color="blue")

    index += 1
    
# Fine-tune figure; make subplots close to each other and hide x ticks for
# all but bottom plot.
f.subplots_adjust(hspace=0.2)
plt.sca(axarr[0, 1])
plt.xticks(rotation=90)
for ax in f.axes:
    mat.pyplot.sca(ax)
    xticks_pos = [0.65*patch.get_width() + patch.get_xy()[0] for patch in h]
    plt.xticks(xticks_pos, age_categories,  ha='right', rotation=45)

# Show the plots!
plt.show()



Okay so we can see here that the age group _30-64 years_ is the highest represented (but also the one spanning the most years, except for _65 and above_, unless we take average length of human lives into account).

Furthermore the distribution looks pretty consistent on all the city districts. There are only slight variations visually noticeable.
E.g. _Nørrebro_ has a lot of people in the _20-29 years_ group compared to _Brønshøj-Husum_ which has a lot fewer. Actually the ratio of _20-29 years_ old is twice as high on _Nørrebro_ than it is in _Brønshøj-Husum_.


In [ ]:
# We'll just create a file here to plot the ratios on the webpage
data = pd.DataFrame(people_in_districts_as_ratio)
data.to_csv("personeriprocent.csv", sep=',', encoding="utf-8")

## Average income by district
We could look at the income by age category as well, but we all probably know how that one will look.

Instead let's take a look at the distribution between the different districts.

This is much like the previous code. Slight alterations have been made to account for the new category we're looking at

In [ ]:
# Create list of categories
income_categories = df.Brutto.unique()

# Three subplots sharing both x/y axes
f, axarr = plt.subplots(5,2, sharey=True, sharex=True, figsize=(10,10))

# Loop through districts and calculate ratio of people living in a district by their age category
index = 0
jj = 0
temp = {}
for district in Districts:
    temp[district] = {}
    income_in_district = df[df['Bydel'].isin([district])].groupby(['Brutto'])['Personer'].sum().astype(float)
    total_people_district = income_in_district.sum()
    for category in income_categories:
        try:
            temp[district][category] = income_in_district[category] / total_people_district
        except KeyError:
            temp[district][category] = 0
    
    d_temp = collections.OrderedDict(sorted(temp[district].items()))
    axarr[index%5, index%2].set_title(district)
    h = axarr[index%5, index%2].bar(range(len(d_temp)), d_temp.values(), color="blue")

    index += 1

# Fine-tune figure; make subplots close to each other and hide x ticks for
# all but bottom plot.
f.subplots_adjust(hspace=0.2)
plt.sca(axarr[0, 1])
plt.xticks(rotation=90)
for ax in f.axes:
    mat.pyplot.sca(ax)
    xticks_pos = [0.65*patch.get_width() + patch.get_xy()[0] for patch in h]
    plt.xticks(xticks_pos, income_categories,  ha='right', rotation=45)

# Show the plots!
plt.show()

At a first glance it looks normally distributed with a positive skew for most of the districts.
There is though a huge difference between the wealthiest and the poorest districts. 
Let's try to see the same chart with only the 3 highest income intervals

In [ ]:
# Create list of categories
income_categories = df.Brutto.unique()[-3:]


# Three subplots sharing both x/y axes
f, axarr = plt.subplots(5,2, sharey=True, sharex=True, figsize=(10,10))

# Loop through districts and calculate ratio of people living in a district by their age category
index = 0
jj = 0
temp = {}
for district in Districts:
    temp[district] = {}
    income_in_district = df[((df.Kategori == 8) 
                   | (df.Kategori == 9)
                   | (df.Kategori == 10))
                   & df['Bydel'].isin([district])].groupby(['Brutto'])['Personer'].sum().astype(float)
    total_people_district = income_in_district.sum()
    for category in income_categories:
        try:
            temp[district][category] = income_in_district[category] / total_people_district
        except KeyError:
            temp[district][category] = 0
    
    d_temp = collections.OrderedDict(sorted(temp[district].items()))
    axarr[index%5, index%2].set_title(district)
    h = axarr[index%5, index%2].bar(range(len(d_temp)), d_temp.values(), color="blue")

    index += 1

# Fine-tune figure; make subplots close to each other and hide x ticks for
# all but bottom plot.
f.subplots_adjust(hspace=0.2)
plt.sca(axarr[0, 1])
plt.xticks(rotation=90)
for ax in f.axes:
    mat.pyplot.sca(ax)
    xticks_pos = [0.65*patch.get_width() + patch.get_xy()[0] for patch in h]
    plt.xticks(xticks_pos, income_categories,  ha='right', rotation=45)

# Show the plots!
plt.show()

# Summing up
Okay so now let's calculate some descriptive statistics about our population.
To do this we must first assume that the data is uniformly distributed within our bucket categories.
We use the following formula to calculate the median
$$ L_m + \left [ \frac { \frac{N}{2} - F_{m-1} }{f_m} \right ] \cdot c $$

And to calculate the mean $\mu$ we simply calculate the mean $\mu_2$ of an interval and use that calculate $\mu$

In [ ]:
people_sum_age = df.groupby(['Aldersgruppe'])['Personer'].sum().astype(float)
people_sum_total = df['Personer'].sum()
age_category_mean = [
    (15+19)/2.0,
    (20+29)/2.0,
    (30+64)/2.0,
    (65+80)/2.0
]
mean_list = [y*x for x,y in zip(age_category_mean, people_sum_age.values)]

mu = (sum(mean_list))/(people_sum_total)
print 'mean for Copenhagen: {0}'.format(mu)

people_sum_district = df.groupby(['Bydel'])['Personer'].sum()
people_age_sum_district = df.groupby(['Bydel', 'Aldersgruppe'])['Personer'].sum()

# calculate mean again

# Decisions, decisions, decisions
Time to look at some decision trees

In [ ]:
from sklearn.feature_extraction import DictVectorizer as DV
from sklearn import tree
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.externals.six import StringIO
import pydot
from IPython.display import Image

df = pd.read_csv(os.path.join(root_directory, "src","data","kon_alder_bydel_penge.csv")) # reads csv
target = df['Bydel']
vectorizer = DV( sparse = False )
train_dict = df[['Kategori', 'Aldersgruppe']].T.to_dict().values()
X = vectorizer.fit_transform(train_dict)
X_train, X_test, target_train, target_test = train_test_split( X, target, test_size=0.1)

clf = tree.DecisionTreeClassifier(max_depth=3)
clf = clf.fit(X_train, target_train)

dot_data = StringIO()
tree.export_graphviz(clf, out_file=dot_data,  
                         feature_names=vectorizer.get_feature_names(),  
                         class_names=target.unique(),  
                         filled=True, rounded=True,
                         special_characters=True)
graph = pydot.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())